In [1]:
import pandas as pd

from mip import Model, xsum, BINARY, INTEGER, OptimizationStatus
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import ipywidgets as widgets

from IPython.display import display, FileLink

import uuid

In [2]:
# remove files from csv folder older that 24 hours
!find 'files/' -name '*.*' -type f -mmin +60 -delete

In [3]:
wL = widgets.BoundedIntText(
    value=6000,
    min=0,
    max=100000.0,
    step=1,
    description='Stock Length:',
    disabled=False
)

myupload = widgets.FileUpload(
    accept='.csv',
    multiple=False  
)

In [4]:
button1 = widgets.Button(description="Optimise")
output = widgets.Output()

def on_button1_clicked(b):
    with output:
        out.clear_output()
        readInCSV()

button1.on_click(on_button1_clicked)

In [5]:
button2 = widgets.Button(description="Try Example CSV")
output = widgets.Output()

def on_button2_clicked(b):
    with output:
        out.clear_output()
        runExampleCSV()

button2.on_click(on_button2_clicked)

In [6]:
button3 = widgets.Button(description="Download Solution")
output = widgets.Output()
    
def on_button3_clicked(b):
            with output:
                createLinkToFiles()                

button3.on_click(on_button3_clicked)

In [7]:
checkbox1 = widgets.Checkbox(
    value=False,
    description='Reduce symmetry',
    disabled=False
)
output = widgets.Output()

# Cutting Stock / One-dimensional Bin Packing Problem

## Upload a CSV file with column names "Quantity" & "Length".  Then click "Optimize".  Enjoy.

For a description see "Cutting Stock / One-dimensional Bin Packing Problem" https://docs.python-mip.com/en/latest/examples.html#cutting-stock-one-dimensional-bin-packing-problem

In [15]:
out = widgets.Output(layout={'border': '1px solid black'})

row0 = widgets.HBox([wL, myupload, button1])
row1 = widgets.HBox([checkbox1])
row2 = widgets.HBox([out])
row3 = widgets.HBox([button2])

widgets.VBox((row0, row1, row2, row3))

[Example CSV File](files/Example.csv)

In [9]:
def readInCSV():
    unique_filename = str(uuid.uuid4())

    uploaded_filename = next(iter(myupload.value))
    content = myupload.value[uploaded_filename]['content']
    
    with open('files/' + unique_filename + '.csv', 'wb') as f: f.write(content)

    url='files/' + unique_filename + '.csv'
    df1=pd.read_csv(url, sep=r'\s*,\s*', engine='python')
    with out:
        print(df1)
        
    !pwd
    !echo $url
    !rm $url

    b = df1['Quantity'].tolist()
    w = df1['Length'].tolist()

    optimization(b, w)
    
def runExampleCSV():
    url='Example.csv'
    df1=pd.read_csv(url, sep=r'\s*,\s*', engine='python')
    with out:
        print(df1)

    b = df1['Quantity'].tolist()
    w = df1['Length'].tolist()

    optimization(b, w)

In [10]:
def optimization(b, w):
    L = wL.value # stock length

    n = round(sum([b[i]*w[i] for i in range(len(b))])/L+1)  # maximum number of bars
    
    m = len(w)  # number of requests

    # creating the model
    model = Model()
    
    x = {(i, j): model.add_var(obj=0, var_type=INTEGER, name="x[%d,%d]" % (i, j))
         for i in range(m) for j in range(n)}
    y = {j: model.add_var(obj=1, var_type=BINARY, name="y[%d]" % j)
         for j in range(n)}

    # constraints
    for i in range(m):
        model.add_constr(xsum(x[i, j] for j in range(n)) >= b[i])
    for j in range(n):
        model.add_constr(xsum(w[i] * x[i, j] for i in range(m)) <= L * y[j])

    # additional constraints to reduce symmetry
    if checkbox1.value == True:
        for j in range(1, n):
            model.add_constr(y[j - 1] >= y[j])
    
    model.emphasis = 1
    
    # optimizing the model
    model.optimize(max_seconds=10, max_solutions=10, relax=False)
    
    # printing the solution
    #print('')
    #print('Objective value: {model.objective_value:.3}'.format(**locals()))
    #print('Solution: ', end='')
    #for v in model.vars:
    #    if v.x > 1e-5:
    #        print('{v.name} = {v.x}'.format(**locals()))
    #        print('          ', end='')
    
    if model.status == OptimizationStatus.OPTIMAL or model.status == OptimizationStatus.FEASIBLE:
        displayOptimization(model.vars, w, L)
    with out:
        print(model.status)    

In [11]:
df = pd.DataFrame()

def displayOptimization(mv, w, L):
    sol = list()
    for v in mv:
        if v.x > 1e-5 and v.name[0] == "x":
            y = (int(v.name.split(',',1)[1].replace(']', ''))) # stock length
            l = (int(v.name.split(',',1)[0].replace('x[', ''))) # cut length
            sol.append((y, int(v.x), w[int(l)]))


    sorted_sol = sorted(sol, key=lambda tup: tup[0])


    # https://stackoverflow.com/questions/61966978/convert-a-list-of-3-tuples-in-to-list-of-list-for-a-matplot-stacked-bar-chart

    dx = max(x[0] for x in sorted_sol) + 1
    predata = []
    for x in range(dx):
        col_data = [tup[1:] for tup in sorted_sol if tup[0] == x]
        temp = [n * [y] for n, y in col_data]
        predata.append([i for sublist in temp for i in sublist])

    dy = max(len(x) for x in predata)
    data = [[i.pop() if i else 0 for i in predata] for _ in range(dy)]
    

    plt.figure()

    global df
    df = pd.DataFrame(data).T
    df = df.loc[(df!=0).any(axis=1)]
    df.plot.bar(legend=False, table=True, stacked=True, title='STOCK LENGTHS')
    plt.tick_params(
        axis='x',          # changes apply to the x-axis
        which='both',      # both major and minor ticks are affected
        bottom=False,      # ticks along the bottom edge are off
        top=False,         # ticks along the top edge are off
        labelbottom=False) # labels along the bottom edge are off
    #plt.xlabel('STOCK LENGTHS')
    plt.ylabel('CUTS REQUIRED IN STOCK LENGTHS')
    with out:
        plt.show()

        #print("Total length")
        #x = (df.sum(axis=1))
        #print(x.sum(axis=0))

        waste=L-df.sum(axis=1)
        waste = waste.rename_axis("Waste for each stock length")
        print(waste)
        print('\n')
        print("Total wasted \t= \t" + str(waste.sum(axis=0)))
        print('% wasted \t= \t'+ '{0:.2f}'.format((waste.sum(axis=0))/(L*dy)*100))
        
        
        display(button3)
           
        


In [12]:
def createLinkToFiles():
    global df

    unique_filename = str(uuid.uuid4())

    df.T.to_csv("files/" + unique_filename + ".csv", encoding='utf-8', index=False)
    local_csv = FileLink("files/" + unique_filename + ".csv", result_html_prefix="Download csv file: ")
    #df.T.to_excel("files/" + unique_filename + ".xlsx", encoding='utf-8', index=False)
    #local_excel = FileLink("files/" + unique_filename + ".xlsx", result_html_prefix="Download excel file: ")
    with out:
        display(local_csv)
        #display(local_excel)